In [1]:
from torch_geometric.data import Data
import network_io as nio
import geopandas as gpd

def extract_subgraph(data, node_indices):
    subgraph_data = data.subgraph(node_indices)
    return subgraph_data

# Abstract

The goal of this notebook is to generate random subgraphs. 

1. Load the network
2. Generate some random subgraphs
3. Visualize the network and the random subgraphs

In [2]:
file_path = '../../../data/pop_1pm/paris_1pm_network.xml.gz'
df = nio.parse_network_xml_gz(file_path)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))
gdf_4326 = gdf.set_crs(epsg=2154).to_crs(epsg=4326)
districts = gpd.read_file("../../../data/visualisation/districts_paris.geojson")

# Parse nodes and edges
nodes_dict = nio.parse_nodes(file_path)
df_edges = nio.parse_edges(file_path, nodes_dict)

# Convert to GeoDataFrame and reproject
gdf_edges = gpd.GeoDataFrame(df_edges, geometry='geometry', crs='EPSG:2154')
gdf_edges_4326 = gdf_edges.to_crs(epsg=4326)

# Spatial join to find districts overlapping with each edge
gdf_edges_with_districts = gpd.sjoin(gdf_edges_4326, districts, how='left', op='intersects')

# Group by edge and aggregate the district names
gdf_edges_with_districts = gdf_edges_with_districts.groupby('id').agg({
    'from': 'first',
    'to': 'first',
    'length': 'first',
    'freespeed': 'first',
    'capacity': 'first',
    'permlanes': 'first',
    'oneway': 'first',
    'modes': 'first',
    'geometry': 'first',
    'c_ar': lambda x: list(x.dropna())
}).reset_index()

# Rename the aggregated column to 'district'
gdf_edges_with_districts.rename(columns={'c_ar': 'district'}, inplace=True)

# Display the resulting GeoDataFrame
gdf_edges_with_districts.head()

/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


,id,from,to,length,freespeed,capacity,permlanes,oneway,modes,geometry,district
0,100315,24972409,24972408,16.18125678991161,8.333333333333334,480.0,1.0,1,"bus,car,car_passenger","LINESTRING (2.33869 48.85181, 2.33847 48.85181)",[6.0]
1,100316,5904976363,24983651,14.860209130036054,8.333333333333334,480.0,1.0,1,"bus,car,car_passenger,pt","LINESTRING (2.33874 48.85242, 2.33872 48.85229)",[6.0]
2,100317,24983651,5904976363,14.860209130036054,8.333333333333334,960.0,2.0,1,"bus,car,car_passenger,pt","LINESTRING (2.33872 48.85229, 2.33874 48.85242)",[6.0]
3,100321,664205947,24972376,22.264539971200314,8.333333333333334,960.0,2.0,1,"car,car_passenger","LINESTRING (2.33994 48.85200, 2.33986 48.85181)",[6.0]
4,100324,24972376,24972375,64.85327605625626,8.333333333333334,480.0,1.0,1,"bus,car,car_passenger","LINESTRING (2.33986 48.85181, 2.33909 48.85152)",[6.0]


In [7]:
# müssen node features und edge index erstellen. 

data = Data(x=x, edge_index=edge_index)

AttributeError: 'DataFrame' object has no attribute 'subgraph'